# Case studies for Robot Test paper - NYC case

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pylab as plt
from matplotlib import rc
rc("text", usetex=True)
rc("font", family="serif")

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)

In [ ]:
t0_day = "2020-04-14"
tests = pd.read_csv("data-nyc/tests_robot_dance.csv", parse_dates=[0])
tests

In [ ]:
basic_prm = prepare_data.save_basic_parameters(min_level=0.8, rep=3.0, ndays=30, window=1)
subnot_factor = 1.0
cities_data, rt_profile = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "NJ", ["NYC"], 10000000, subnot_factor, 1, "data-nyc/nyc_robot_data.csv", t0_day=t0_day, tests=tests["tests"].values)
cities_data

In [ ]:
rt_profile

In [ ]:
tests_profile = tests[tests["date"] >= pd.to_datetime(t0_day)]
tests_profile = tests_profile["tests"].values
tests_profile

In [ ]:
# Define the basic data for the case studies

# Basic reproduction number
basic_rep = 3.0

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = len(rt_profile)

print(ndays, len(rt_profile), len(tests_profile))

# Mean time in ICU
time_icu = 7

# Lockdown level
lock_level = 0.6

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(min_level=lock_level, rep=basic_rep, time_icu=time_icu, ndays=ndays)

# Compute initial values

# For DRS
cities_data, rt_profile = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "NJ", [], 000000, subnot_factor, 1, "data-nyc/nyc_robot_data.csv", t0_day="2020-04-14", tests=tests["tests"].values)
cities_data

In [ ]:
t = [0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.9179991083247093,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,0.8257716552168826,3.0,3.0,3.0]
len(t)

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.ones((ncities, ndays))
cities_data

## Add information on the time series that estimate the need of ICUs

We are using the time series adjusted considering that the mean ICU stay is 7 days (which lead to larger ICU capacity).

In [ ]:
if basic_prm["time_icu"] == 7:
    # Time series adjusted considering the mean ICU time is 7 days
    ts = np.array([0.00857702, 0.01956817, 0.00180512, 0.0, 1.97780974, -0.9797225, np.sqrt(6.326E-05), 0.009785159188786, 0.010250436563906])
else:
    raise NotImplementedError

ts_drs = np.ones((len(cities_data), len(ts)))
ts_drs *= ts
ts_drs = pd.DataFrame(data=ts_drs, index=cities_data.index, columns=[
    "rho_min", "rho_max", "intercept", "trend", "phi_1", "phi_2", "sigma_omega", "state0", "state_less_1"
])
ts_drs["confidence"] = 0.9
ts_drs["time_icu"] = time_icu
cities_data = pd.concat([cities_data, ts_drs], axis=1)
cities_data


In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, pools=None, budget=0, tests_off=np.zeros(0, int), tau=3, test_efficacy=0.8, daily_tests=0, proportional_tests=False, tests=np.zeros(0), fixed_rt=np.zeros(0), verbosity=1):
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif, pools, 
        verbosity=verbosity, test_budget=budget, tests_off=tests_off, tau=tau, test_efficacy=test_efficacy, 
        daily_tests=daily_tests, proportional_tests=proportional_tests, tests=tests, fixed_rt=fixed_rt)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, target, verbosity=verbosity)
    

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=1, drs="data-nyc/mobility_matrix.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
# run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
#     incr_all=True, verbosity=1)
print("Hammer data", hammer_data)
M.loc["NYC", "NYC"]


In [ ]:
all_tau = [2] #[1, 2, 3, 4, 5]
all_test_efficacy = [0.8] #[0.7, 0.8, 0.9]
all_daily_tests = [30000] #[30000] , 50000]
all_budgets = [20000000] #[0, 5000000, 10000000, 20000000]

## Test experiments

### 1 Fix rt and tests as performed

In [ ]:
tau, test_efficacy = 2, 0.8
daily_tests, budget = 0, 0

print("******************** Running tau =", tau, "test_effic =", test_efficacy, "daily tests =", daily_tests, "budget =", budget)
print("******************** Test fixed on original values")

# Case 1 - Test fixed
basic_prm["alternate"] = 0.0
tests_off = np.zeros(0, int)
result_file =  f"results/original_tests_nyc.csv"
figure_file = f"results/original_tests_nyc.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests, False, tests=tests_profile, fixed_rt=rt_profile);


In [ ]:
tau, test_efficacy = 2, 0.8
daily_tests, budget = 0, 0

print("******************** Running tau =", tau, "test_effic =", test_efficacy, "daily tests =", daily_tests, "budget =", budget)
print("******************** Test with zero tests")

# Case 1 - Test fixed
basic_prm["alternate"] = 0.0
tests_off = np.zeros(0, int)
result_file =  f"results/no_tests_nyc.csv"
figure_file = f"results/no_tests_nyc.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, 2.0*target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests, False, tests=np.zeros(0), fixed_rt=rt_profile);

In [ ]:
tau, test_efficacy = 2, 0.8
pop = cities_data.loc["NYC", "population"]
total_tests = sum(tests_profile)*pop
daily_tests, budget = total_tests, total_tests

print("******************** Running tau =", tau, "test_effic =", test_efficacy, "daily tests =", daily_tests, "budget =", budget)
print("******************** Test with zero tests")

# Case 1 - Test fixed
basic_prm["alternate"] = 0.0
tests_off = np.zeros(0, int)
result_file =  f"results/optimal_tests_nyc.csv"
figure_file = f"results/optimal_tests_nyc.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, 2.0*target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests, False, tests=np.zeros(0), fixed_rt=rt_profile);

In [ ]:
tau, test_efficacy = 2, 0.8
pop = cities_data.loc["NYC", "population"]
total_tests = sum(tests_profile)*pop
daily_tests, budget = total_tests, 10*total_tests

print("******************** Running tau =", tau, "test_effic =", test_efficacy, "daily tests =", daily_tests, "budget =", budget)
print("******************** Test with zero tests")

# Case 1 - Test fixed
basic_prm["alternate"] = 0.0
tests_off = np.zeros(0, int)
result_file =  f"results/optimal_tests_extra_budget_nyc.csv"
figure_file = f"results/optimal_tests_extra_budget_nyc.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, 2.0*target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests, False, tests=np.zeros(0), fixed_rt=rt_profile);